# Projet F422


## Import


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [5]:
path = "F422 EMG project data"

X_guided_train = np.load(f"{path}/guided/guided_dataset_X.npy")
y_guided_train = np.load(f"{path}/guided/guided_dataset_y.npy")
X_guided_test = np.load(f"{path}/guided/guided_testset_X.npy")

X_freemoves_train = np.load(f"{path}/freemoves/freemoves_dataset_X.npy")
y_freemoves_train = np.load(f"{path}/freemoves/freemoves_dataset_y.npy")
X_freemoves_test = np.load(f"{path}/freemoves/freemoves_testset_X.npy")

In [6]:
print("Guid:")
print(f"X_train {X_guided_train.shape}")
print(f"y_train {y_guided_train.shape}") 
print(f"X_test {X_guided_test.shape}\n")

print("Free:")
print(f"X_train {X_freemoves_train.shape}")
print(f"y_train {y_freemoves_train.shape}")
print(f"X_test {X_freemoves_test.shape}")

Guid:
X_train (5, 8, 230000)
y_train (5, 51, 230000)
X_test (5, 332, 8, 500)

Free:
X_train (5, 8, 270000)
y_train (5, 51, 270000)
X_test (5, 308, 8, 500)


## Question 1 (bonus)  : Filtre

In [8]:
def filtre(data, size=5):
    filtered = np.zeros_like(data)
    for session in range(data.shape[0]):
        for electrode in range(data.shape[1]):
            x = data[session, electrode]
            x_pad = np.pad(x, (size//2,), mode='edge')  # padding pour conserver la taille
            x_smooth = np.convolve(x_pad, np.ones(size)/size, mode='valid')
            filtered[session, electrode] = x_smooth
    return filtered

## Question 2 : Overlap

In [10]:
def overlap(data, overlap=0.5, size=500):
    Data = []
    step = int(size * (1 - overlap))
    n = (data.shape[2] - size) // step + 1
    fin = n * step
    
    for start in range(0, fin, step):
        end = start + size
        W = data[... , start:end]
        Data.append(W)
        
    Data = np.array(Data)
    Data = Data.transpose(1, 0, 2, 3)
    return Data

In [11]:
X = overlap(X_guided_train)
Y = overlap(y_guided_train)

## Question 3 : Cross Validation K-Fold

In [13]:
def redimensionne(X, Y):

    x = X.reshape(-1, X.shape[2], X.shape[3])  
    y = Y.reshape(-1, Y.shape[2], Y.shape[3])  

    vecX = x.reshape(x.shape[0], -1)  
    vecY = y.mean(axis=-1)  
    # vecY = y.reshape(y.shape[0], -1)
    
    return vecX, vecY


In [14]:
vecX, vecY = redimensionne(X, Y)
K=5
kf = KFold(n_splits=K)

for train_id, val_id in kf.split(vecX):
    X_train, X_val = vecX[train_id], vecX[val_id]
    Y_train, Y_val = vecY[train_id], vecY[val_id]

    
    #model = ... #chosir model
    #model.fit(X_train, y_train)

    #evaluation des resultat
